In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

In [ ]:
import tensorflow as tf
print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.12.0


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_data_input_dir = "/content/gdrive/MyDrive/Work/BCA/train_data/train_data_PT_cabang_237.csv"
model_output_dir = "/content/gdrive/MyDrive/Work/BCA/models/model_LSTM_PT_cabang_237_ver_2"

In [ ]:
train_data = pd.read_csv(train_data_input_dir, header=0, sep="|")

In [ ]:
train_data

,dt,sumtrx
0,2020-01-02,28
1,2020-01-03,26
2,2020-01-04,0
3,2020-01-05,0
4,2020-01-06,41
...,...,...
1259,2023-06-14,48
1260,2023-06-15,44
1261,2023-06-16,44
1262,2023-06-17,16


In [ ]:
# Membuat fungsi untuk mengubah data menjadi bentuk yang sesuai untuk LSTM
def create_dataset(data, time_steps):
    X, Y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        Y.append(data[i + time_steps, 0])
    return np.array(X), np.array(Y)

In [ ]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   print(
#       '\n\nThis error most likely means that this notebook is not '
#       'configured to use a GPU.  Change this in Notebook Settings via the '
#       'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
#   raise SystemError('GPU device not found')

In [ ]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
# Mengubah data menjadi skala antara 0 dan 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_data = scaler.fit_transform(train_data["sumtrx"].values.reshape(-1, 1))
# scaled_test_data = scaler.transform(.values.reshape(-1, 1))
# Menentukan jumlah time steps
time_steps = 7

# Membuat dataset latih dan uji
X_train, y_train = create_dataset(scaled_train_data, time_steps)
# X_test, y_test = create_dataset(scaled_test_data, time_steps)

# Reshape input menjadi bentuk yang sesuai untuk LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Membangun model LSTM
model = Sequential()
model.add(LSTM(60, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(60))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Melatih model
# with tf.device('/device:GPU:0'):
history = model.fit(X_train, y_train, epochs=8000, batch_size=64)
# Melakukan prediksi
# predictions = model.predict(X_test)
# predictions = scaler.inverse_transform(predictions)

# actual_vs_predicted[branch] = pd.DataFrame(
#     data={
#         "Actual": test_data_for_branch[branch].iloc[:len(predictions), 0],
#         "Predicted": predictions[:, 0].squeeze()
#     }
# )

model.save(model_output_dir)



Output streaming akan dipotong hingga 5000 baris terakhir.
20/20 [==============================] - 0s 9ms/step - loss: 1.0618e-06
Epoch 5502/8000
20/20 [==============================] - 0s 9ms/step - loss: 7.9869e-07
Epoch 5503/8000
20/20 [==============================] - 0s 9ms/step - loss: 6.3402e-07
Epoch 5504/8000
20/20 [==============================] - 0s 10ms/step - loss: 5.9507e-07
Epoch 5505/8000
20/20 [==============================] - 0s 8ms/step - loss: 4.8852e-07
Epoch 5506/8000
20/20 [==============================] - 0s 9ms/step - loss: 4.4135e-07
Epoch 5507/8000
20/20 [==============================] - 0s 9ms/step - loss: 4.3887e-07
Epoch 5508/8000
20/20 [==============================] - 0s 9ms/step - loss: 3.5723e-07
Epoch 5509/8000
20/20 [==============================] - 0s 9ms/step - loss: 3.3775e-07
Epoch 5510/8000
20/20 [==============================] - 0s 9ms/step - loss: 3.4945e-07
Epoch 5511/8000
20/20 [==============================] - 0s 9ms/step - loss:

In [ ]:
# temp_actual_vs_predicted = {}

# for branch in list(train_data_for_branch):
#   temp_actual_vs_predicted[branch] = pd.DataFrame(
#       data={
#           "Actual": test_data_for_branch[branch].iloc[:actual_vs_predicted[branch].shape[0], 0],
#           "Predicted": actual_vs_predicted[branch]["Predicted"]
#       }
#   )

In [ ]:
# actual_vs_predicted = temp_actual_vs_predicted

In [ ]:
from math import sqrt
from sklearn.metrics import mean_absolute_error

In [ ]:
def calc_rmse(actual, predicted):
  rmse = sqrt(mean_squared_error(actual, predicted))
  return rmse

In [ ]:
def calc_mae(actual, predicted):
  mae = mean_absolute_error(actual, predicted)
  return mae

In [ ]:
def calc_smape(actual, predicted):
    numerator = np.abs(actual - predicted)
    denominator = (np.abs(actual) + np.abs(predicted)) / 2
    smape_score = np.mean(numerator / denominator) * 100
    return smape_score

In [ ]:
def evaluate(actual_vs_predicted):
  plt.figure(figsize=(18, 6))
  plt.plot(actual_vs_predicted.index, actual_vs_predicted["Actual"])
  plt.plot(actual_vs_predicted.index, actual_vs_predicted["Predicted"])

  # Mengatur label dan judul
  plt.xlabel('Date')
  plt.ylabel('sumtrx')
  plt.title('Actual vs Predicted')

  print("Root Mean Squared Error (RMSE):", calc_rmse(actual_vs_predicted["Actual"], actual_vs_predicted["Predicted"]))
  print("Mean Absolute Error (MAE):", calc_mae(actual_vs_predicted["Actual"], actual_vs_predicted["Predicted"]))
  print("SMAPE score:", calc_smape(actual_vs_predicted["Actual"], actual_vs_predicted["Predicted"]))

In [ ]:
# evaluate(actual_vs_predicted[1130])

In [ ]:
# evaluate(actual_vs_predicted[14])

In [ ]:
# evaluate(actual_vs_predicted[21])

In [ ]:
# evaluate(actual_vs_predicted[64])

In [ ]:
# evaluate(actual_vs_predicted[8])